2/9 요약정리

과적합이 생길때 규제를 통해 회귀계수를 줄어야 나가고, 결정계수도 줄어나간다?
1) L2 규제
2) L1 규제

하이퍼파리미터 => 분석가가 결정(하이퍼파라미터 튜닝)
cross validation score

엘라스틱넷
L2 + L1 => 하이퍼파라미터가 갯수가 2개 이상
1) grid -> 최적의 하이퍼파라미터를 찾고 결정, GridSearchCV

LosticRegression
1) 결정(0, 1)을 하기위해 여러가지 특성 데이터를 준비
2) 시그모이드 함수
3) 임계치값을 기준으로 최종결정(0,1)이 되어진다. -> 임계치를 설정하면 데이터조작이 가능하다
4) 최종결정(=예측값)이 얼마나 잘 설명하는가 => 분류 성능 지표
    - 오차행렬(confusion_matrix)
    - 정확도, 재현율, 정밀도, f1_score(조화평균), ROC-AUC

* 코드
'''
lr.predict(X_test)      => 이산값(int)
lr.predict.prob(X_test) => 확률값(float)
'''


# 로지스틱회귀

1. 다중 분류 실습
    - 

# [실습] kaggle 피마 인디언 당뇨병 데이터를 분석하여 당뇨병 여부를 예측하세요

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, precision_recall_curve, roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

diabetes_data = pd.read_csv('/Users/kimminsoo/Desktop/ML/data/pima_project/train.csv')
print(diabetes_data['Outcome'].value_counts())
diabetes_data.head(3)

0    400
1    214
Name: Outcome, dtype: int64


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,7,109.0,80.0,31.0,79.799479,35.9,1.127,43,1
1,4,129.0,86.0,20.0,270.000000,35.1,0.231,23,0
2,2,123.0,48.0,32.0,165.000000,42.1,0.520,26,0


- Pima indians: 북미 피마지역 원주민 type2 당뇨병 결과 데이터, 고립된 지역의 고유 혈통을 지닌 - - 집단에 서구 식습관이 당뇨에 미친 영향
- pregnancies: 임신 횟수
- glucose: 포도당 부하 검사 수치
- bloodpressure : 혈압(mm Hg)
- skinthickness : 팔 삼두근 뒤쪽의 피하지방 측정값(mm)
- insulin: 혈청 인슐린(mu U/ml)
- BMI:  체질량지수( 체중(kg) / 키(m)**2)
- DiabetesPedigreeFuncions: 당뇨 내력 가중치 값
- Age : 나이
- Outcome: 클래스 결정값(0, 1)

[pima datast 링크](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database)

In [72]:
diabetes_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               614 non-null    int64  
 1   Glucose                   614 non-null    float64
 2   BloodPressure             614 non-null    float64
 3   SkinThickness             614 non-null    float64
 4   Insulin                   614 non-null    float64
 5   BMI                       614 non-null    float64
 6   DiabetesPedigreeFunction  614 non-null    float64
 7   Age                       614 non-null    int64  
 8   Outcome                   614 non-null    int64  
dtypes: float64(6), int64(3)
memory usage: 43.3 KB


In [73]:
diabetes_data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,614.000,614.000,614.000,614.000,614.000,614.000,614.000,614.000,614.000
mean,3.746,121.885,71.909,26.624,117.378,32.412,0.472,32.888,0.349
std,3.323,30.557,12.172,9.933,89.553,6.866,0.333,11.565,0.477
min,0.000,56.000,24.000,7.000,14.000,18.200,0.078,21.000,0.000
25%,1.000,100.000,64.000,20.536,79.799,27.500,0.239,24.000,0.000
50%,3.000,117.000,70.000,23.000,79.799,32.000,0.372,29.000,0.000
75%,6.000,140.000,80.000,32.000,127.750,36.500,0.613,40.000,1.000
max,17.000,199.000,122.000,99.000,846.000,67.100,2.420,81.000,1.000


In [92]:
question = pd.read_csv('/Users/kimminsoo/Desktop/ML/data/pima_project/test.csv')
question

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,2,74.000,69.105,20.536,79.799,31.993,0.102,22
1,0,95.000,64.000,39.000,105.000,44.600,0.366,22
2,3,111.000,90.000,12.000,78.000,28.400,0.495,29
3,7,129.000,68.000,49.000,125.000,38.500,0.439,43
4,8,125.000,96.000,20.536,79.799,31.993,0.232,54
...,...,...,...,...,...,...,...,...
149,1,128.000,48.000,45.000,194.000,40.500,0.613,24
150,8,95.000,72.000,20.536,79.799,36.800,0.485,57
151,1,91.000,54.000,25.000,100.000,25.200,0.234,23
152,3,173.000,84.000,33.000,474.000,35.700,0.258,22


In [93]:
question.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               154 non-null    int64  
 1   Glucose                   154 non-null    float64
 2   BloodPressure             154 non-null    float64
 3   SkinThickness             154 non-null    float64
 4   Insulin                   154 non-null    float64
 5   BMI                       154 non-null    float64
 6   DiabetesPedigreeFunction  154 non-null    float64
 7   Age                       154 non-null    int64  
dtypes: float64(6), int64(2)
memory usage: 9.8 KB


## 1) Foldout 검증

In [110]:
X = diabetes_data.drop('Outcome', axis=1).values
y = diabetes_data.Outcome

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

In [111]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [112]:
pd.options.display.float_format = '{:.3f}'.format
pd.DataFrame(X_train_scaled).describe()

,0,1,2,3,4,5,6,7
count,491.000,491.000,491.000,491.000,491.000,491.000,491.000,491.000
mean,-0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,-0.000
std,1.001,1.001,1.001,1.001,1.001,1.001,1.001,1.001
min,-1.095,-2.128,-3.925,-1.956,-1.088,-2.094,-1.196,-1.018
25%,-0.796,-0.737,-0.634,-0.607,-0.419,-0.720,-0.701,-0.757
50%,-0.198,-0.164,-0.140,-0.362,-0.419,-0.045,-0.299,-0.322
75%,0.699,0.590,0.518,0.535,0.070,0.624,0.445,0.548
max,3.987,2.522,4.139,7.209,7.874,3.371,5.637,3.245


In [125]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='liblinear')

lr.fit(X_train_scaled, y_train)

y_pred = lr.predict(X_test_scaled)
y_pred

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0])

In [126]:
np.set_printoptions(suppress=True)
y_pred_prob = lr.predict_proba(X_test_scaled)
y_pred_prob

array([[0.1130569 , 0.8869431 ],
       [0.66401401, 0.33598599],
       [0.61210828, 0.38789172],
       [0.55587516, 0.44412484],
       [0.70679006, 0.29320994],
       [0.87816033, 0.12183967],
       [0.71455971, 0.28544029],
       [0.87820601, 0.12179399],
       [0.78078619, 0.21921381],
       [0.33400286, 0.66599714],
       [0.85028679, 0.14971321],
       [0.69810661, 0.30189339],
       [0.97272994, 0.02727006],
       [0.83999878, 0.16000122],
       [0.96670995, 0.03329005],
       [0.35801536, 0.64198464],
       [0.88568379, 0.11431621],
       [0.99038504, 0.00961496],
       [0.60258508, 0.39741492],
       [0.75244702, 0.24755298],
       [0.79578258, 0.20421742],
       [0.46529297, 0.53470703],
       [0.92767397, 0.07232603],
       [0.71597285, 0.28402715],
       [0.4604023 , 0.5395977 ],
       [0.81880769, 0.18119231],
       [0.03882933, 0.96117067],
       [0.37608708, 0.62391292],
       [0.70456751, 0.29543249],
       [0.71712585, 0.28287415],
       [0.

In [127]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[62, 12],
       [24, 25]])

In [132]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)
acc

0.7073170731707317

In [133]:
question

q_test = scaler.transform(question)
q_pred = lr.predict(q_test)

q_pred

/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:413: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


array([0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0])

In [134]:
question_df = pd.DataFrame(q_pred)
question_df

,0
0,0
1,0
2,0
3,1
4,0
...,...
149,1
150,0
151,0
152,1


In [137]:
submission['Outcome'] = question_df
submission

,No,Outcome
0,1,0
1,2,0
2,3,0
3,4,1
4,5,0
...,...,...
149,150,1
150,151,0
151,152,0
152,153,1


In [138]:
submission.to_csv('kms.csv', index=False)

## 2) KFold 검증

In [140]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

X = diabetes_data.drop('Outcome', axis=1).values
y = diabetes_data.Outcome

kfold = KFold(7)
lr = LinearRegression()

cross_val_score(lr,X,y, cv=7)
np.round(np.mean(cross_val_score(lr,X,y, cv=7)),3)

(array([0.32018201, 0.27603291, 0.27201329, 0.37398091, 0.26415248,
        0.27345218, 0.21190871]),
 0.285)

### submission 읽어오기

In [108]:
submission = pd.read_csv('/Users/kimminsoo/Desktop/ML/data/pima_project/submission.csv')

In [109]:
submission.to_csv('kms.csv', index=False)

### submission 저장하기(index=False)